In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pandas as pd
import pickle
import numpy as np

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append("../exp_adaptive_sv/")

In [8]:
from utils import get_id2idx
si_embed_dir = "embeddings/voxc2_fbank64_voxc2untied_xvector/"
si_embeds = np.load(si_embed_dir + "/si_embeds.npy")
si_keys = pickle.load(open(si_embed_dir + "/si_keys.pkl", "rb"))
si_id2idx = get_id2idx(si_keys)

sv_embed_dir = "embeddings/voxc2_fbank64_voxc2untied_xvector/"
sv_embeds = np.load(sv_embed_dir + "/sv_embeds.npy")
sv_keys = pickle.load(open(sv_embed_dir + "/sv_keys.pkl", "rb"))
sv_id2idx = get_id2idx(sv_keys)

# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# si_embed_mean = si_embeds.mean(0)
# centered_si_embeds = si_embeds - si_embed_mean.reshape(1, -1)
# centered_sv_embeds = sv_embeds - si_embed_mean.reshape(1, -1)

# clf = LDA(solver='svd', n_components=200)
# clf.fit(centered_si_embeds, si_df.label)

# lda_si_embeds = clf.transform(centered_si_embeds)
# lda_sv_embeds = clf.transform(centered_sv_embeds)

In [11]:
from utils import key2df
si_df = key2df(si_keys)
sv_df = key2df(sv_keys)

In [10]:
plda_dir = sv_embed_dir + "plda_train/"

# Load Data.

In [8]:
lda_si_embeds = np.load("embeddings/voxc2_fbank64_voxc2untied_xvector/lda_si_embeds.npy")
lda_sv_embeds = np.load("embeddings/voxc2_fbank64_voxc2untied_xvector/lda_sv_embeds.npy")

In [47]:
# length normalization
ln_lda_si_embeds = lda_si_embeds * np.sqrt(lda_si_embeds.shape[1]) / \
                                           np.linalg.norm(lda_si_embeds, axis=1, keepdims=True)
ln_lda_sv_embeds = lda_sv_embeds * np.sqrt(lda_sv_embeds.shape[1]) / \
                                           np.linalg.norm(lda_sv_embeds, axis=1, keepdims=True)

In [60]:
np.save("embeddings/voxc2_fbank64_voxc2untied_xvector/ln_lda_si_embeds.npy", ln_lda_si_embeds)

In [61]:
np.save("embeddings/voxc2_fbank64_voxc2untied_xvector/ln_lda_sv_embeds.npy", ln_lda_sv_embeds)

## PLDA Verifier 

In [95]:
from ioffe_plda.verifier import Verifier
saved_model = pickle.load(open("verifier_lda_embeds.pkl", "rb"))
verifier = Verifier(saved_model)

In [72]:
# verifier.fit(si_embeds, si_df.label)
# pickle.dump(verifier, open("verifier_tmp.pkl", "wb"))

In [85]:
%%time
py_plda_scores = verifier.multi_sess(ln_lda_sv_embeds[:20], ln_lda_sv_embeds[20:])

CPU times: user 29 s, sys: 30.8 s, total: 59.8 s
Wall time: 1.42 s


In [104]:
%%time
py_plda_scores = verifier.score_avg(ln_lda_sv_embeds[:20], ln_lda_sv_embeds[20:])

CPU times: user 3min 46s, sys: 3min 42s, total: 7min 29s
Wall time: 13.2 s


In [106]:
%%time
py_plda_scores = verifier.vector_avg(ln_lda_sv_embeds[:20], ln_lda_sv_embeds[20:])

CPU times: user 27.6 s, sys: 35.9 s, total: 1min 3s
Wall time: 1.97 s


In [54]:
from batch_sv_system_utils import compute_plda_score
kaldi_plda_scores = compute_plda_score(sv_embeds[:20], sv_embeds[20:], plda_dir, mean=True)

In [52]:
labels = np.array(sv_df[20:].label == 0, dtype=np.int64)

In [107]:
from batch_sv_system_utils import compute_eer
compute_eer(np.round(py_plda_scores.mean(0), 5), labels)

(0.01754385964912286, 0.017335879404844923, 0.01754385964912286, 8.09347)

In [56]:
from batch_sv_system_utils import compute_eer
compute_eer(np.round(kaldi_plda_scores[0], 5), labels)

(0.018117949152431912, 0.018117949152431912, 0.01754385964912286, -32.43853)